<a href="https://colab.research.google.com/github/anandsai410/Blockchain-technologies-Assignment-I/blob/main/ORT_Manual_Global_2503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import nltk
import datetime
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import punkt
import openpyxl
import pandas as pd
import numpy as np
import re
from datetime import datetime, date, time
import time
from dateutil.parser import parser
from dateutil.relativedelta import relativedelta
parser=parser()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [105]:
!pip install spark-nlp
!pip install pyspark

import sparknlp
from sparknlp.annotator import DocumentAssembler, DateMatcher, MultiDateMatcher
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline

spark = sparknlp.start()
spark

In [116]:
df = pd.read_excel("Global25-03-24.xlsx")
df

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Number,External Info,ORT manual,Manager,Assigned to,State,Priority,Next SLA due date,Next SLA reason,Tags,Updated by,Customer contact
0,CS20240007061863,2024-03-22 21:54:08 - Aaron Cooke (I565751) (E...,2024-03-25 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),Awaiting Info,2 - High,2024-03-25 22:30:00,ORT (SLA),NaN,S0023855138,Swapna Shetty S (S0021840755)
1,CS20240007164010,2024-03-25 14:51:03 - Aaron Cooke (I565751) (E...,2024-03-26 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,2 - High,2024-03-26 22:30:00,ORT (SLA),NaN,system,Arun Kumara Y A (S0018363415)
2,CS20240007084283,2024-03-23 01:29:03 - Lovely Barba (I349243) (...,2024-03-27 03:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,1 - Very High,2024-03-27 03:30:00,ORT (SLA),NaN,I545965,marychu billones (S0025814704)
3,CS20230006469172,2024-03-22 18:58:47 - Aaron Cooke (I565751) (E...,2024-03-27 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,3 - Medium,2024-03-27 22:30:00,ORT (SLA),"#PFA_90+_Bustathon_2024, #PL_HELP_Tobias, #Q1_...",system,Lisa Dickie (S0025121929)
4,CS20240007189121,2024-03-21 15:03:22 - Aaron Cooke (I565751) (E...,2024-03-27 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,2 - High,2024-03-27 22:30:00,ORT (SLA),NaN,system,Erin Dick (S0014020613)
...,...,...,...,...,...,...,...,...,...,...,...,...
462,CS20230006603345,2024-02-16 06:01:05 - Youssouf Sangare (I31938...,2024-03-10 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,3 - Medium,2024-03-10 05:29:56,ORT (SLA),"#PFA_90+_Bustathon_2024, #Q1_2024_Backlog_Trac...",S0023223396,Ulrike Stadler (S0023223396)
463,CS20240007008827,2024-03-18 07:01:34 - Youssouf Sangare (I31938...,2024-04-06 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,4 - Low,2024-04-06 05:29:56,ORT (SLA),NaN,system,Nina Hynynen (S0021536958)
464,CS20240007270560,2024-03-23 02:23:25 - Youssouf Sangare (I31938...,2024-04-06 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,2 - High,2024-04-06 05:29:56,ORT (SLA),NaN,system,Ahmed haridy (S0024058876)
465,CS20240006832810,2024-03-20 18:42:14 - Chloe Doyle (I530079) (E...,2024-04-01 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,3 - Medium,2024-04-01 05:29:56,ORT (SLA),#Q1_2024_Backlog_Tracking,I530079,Michelle McCann (S0025036898)


In [117]:
ext_inf=df['External Info']
dte=df['ORT manual']
df['CP_ext_inf']=''
df['OA_ext_inf']=''
df['dates']=''
df['check']=''

In [118]:
pattern_SE = r'\<p>(.*?)\</p>'
pattern_OA = r'\<br/>(.*?)\<br/>'

for i in range(0,len(df)):
  xstr=""
  ystr=""
  x = re.findall(pattern_SE, df.iloc[i,1])
  y = re.findall(pattern_OA, df.iloc[i,1])
  for j in x:
     xstr=xstr+". "+str(j)
  for j in y:
     ystr = ystr+". "+str(j)
  df.iloc[i,12]=xstr
  df.iloc[i,13]=ystr

In [119]:
df

,Number,External Info,ORT manual,Manager,Assigned to,State,Priority,Next SLA due date,Next SLA reason,Tags,Updated by,Customer contact,CP_ext_inf,OA_ext_inf,dates,check
0,CS20240007061863,2024-03-22 21:54:08 - Aaron Cooke (I565751) (E...,2024-03-25 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),Awaiting Info,2 - High,2024-03-25 22:30:00,ORT (SLA),NaN,S0023855138,Swapna Shetty S (S0021840755),. Hi there. Investigation is still continuing ...,. You will receive an update about this incide...,,
1,CS20240007164010,2024-03-25 14:51:03 - Aaron Cooke (I565751) (E...,2024-03-26 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,2 - High,2024-03-26 22:30:00,ORT (SLA),NaN,system,Arun Kumara Y A (S0018363415),. Hi Arun. Thank you for this - the colleague ...,. You will receive an update about this incide...,,
2,CS20240007084283,2024-03-23 01:29:03 - Lovely Barba (I349243) (...,2024-03-27 03:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,1 - Very High,2024-03-27 03:30:00,ORT (SLA),NaN,I545965,marychu billones (S0025814704),". Hi Marychu,. Good day!. My name is Lovely an...",. . Contact phone or email: +95-17301589.,,
3,CS20230006469172,2024-03-22 18:58:47 - Aaron Cooke (I565751) (E...,2024-03-27 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,3 - Medium,2024-03-27 22:30:00,ORT (SLA),"#PFA_90+_Bustathon_2024, #PL_HELP_Tobias, #Q1_...",system,Lisa Dickie (S0025121929),. Hi there. I have brought to product lead tod...,. . Channel: Call from customer. Interaction s...,,
4,CS20240007189121,2024-03-21 15:03:22 - Aaron Cooke (I565751) (E...,2024-03-27 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,2 - High,2024-03-27 22:30:00,ORT (SLA),NaN,system,Erin Dick (S0014020613),. Hi Erin. The colleague who I was advised to ...,. You will receive an update about this incide...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,CS20230006603345,2024-02-16 06:01:05 - Youssouf Sangare (I31938...,2024-03-10 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,3 - Medium,2024-03-10 05:29:56,ORT (SLA),"#PFA_90+_Bustathon_2024, #Q1_2024_Backlog_Trac...",S0023223396,Ulrike Stadler (S0023223396),". &nbsp;Hello Sabrina ,. your feedback are und...",. . Contact phone or email: +353-831817719.,,
463,CS20240007008827,2024-03-18 07:01:34 - Youssouf Sangare (I31938...,2024-04-06 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,4 - Low,2024-04-06 05:29:56,ORT (SLA),NaN,system,Nina Hynynen (S0021536958),". Hi Nina,. we are working on your feedback an...",,,
464,CS20240007270560,2024-03-23 02:23:25 - Youssouf Sangare (I31938...,2024-04-06 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,2 - High,2024-04-06 05:29:56,ORT (SLA),NaN,system,Ahmed haridy (S0024058876),". Hello &nbsp;Ahmed &nbsp; ,&nbsp;. Call feedb...",. . Contact phone or email: +20-01030554221.,,
465,CS20240006832810,2024-03-20 18:42:14 - Chloe Doyle (I530079) (E...,2024-04-01 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,3 - Medium,2024-04-01 05:29:56,ORT (SLA),#Q1_2024_Backlog_Tracking,I530079,Michelle McCann (S0025036898),. <b>Requester name:</b> Roy SAP<br /><b>Reque...,. . Channel: Call from customer. Interaction s...,,


In [120]:
def is_date(string):
  if ((string[0:4]+string[5:7]+string[8:10]).isdigit()) or ((string[0:2]+string[3:5]+string[6:10]).isdigit()):
    return True
  else:
    return False

def convert_dte(x):
  if (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="/" and int(x[0:2])<=12 and int(x[3:5])>12:
    y=datetime.strptime(x, "%m/%d/%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="." and int(x[0:2])<=12 and int(x[3:5])>12:
    y=datetime.strptime(x, "%m.%d.%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="-" and int(x[0:2])<=12 and int(x[3:5])>12:
    y=datetime.strptime(x, "%m-%d-%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:4]+x[5:7]+x[8:10]).isdigit() and x[4]=="-":
    y=datetime.strptime(x, "%Y-%m-%d")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="-":
    y=datetime.strptime(x, "%d-%m-%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:4]+x[5:7]+x[8:10]).isdigit() and x[4]==".":
    y=datetime.strptime(x, "%Y.%m.%d")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]==".":
    y=datetime.strptime(x, "%d.%m.%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:4]+x[5:7]+x[8:10]).isdigit() and x[4]=="/":
    y=datetime.strptime(x, "%Y/%m/%d")
    y=y.strftime("%Y.%m.%d")
    return y
  elif (x[0:2]+x[3:5]+x[6:10]).isdigit() and x[2]=="/":
    y=datetime.strptime(x, "%d/%m/%Y")
    y=y.strftime("%Y.%m.%d")
    return y
  else:
    return ''

In [121]:
# Step 1: Transforms raw texts to `document` annotation
document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)
# Step 2: Extracts one date information from text
date = (
    DateMatcher()
    .setInputCols("document") \
    .setOutputCol("date") \
    .setOutputFormat("yyyy.MM.dd")
)

pipl = Pipeline(stages=[document_assembler, date])

for i in range(0,len(df)-1):
  dtestr=""

  #ORT Assistant part
  text_oa = word_tokenize(df.iloc[i,13])
  for j in text_oa:
    if is_date(j):
      try:
        j=convert_dte(j)
      except (IndexError, ValueError):
        pass
      dtestr=dtestr+str(j)+", "
  df.iloc[i,14] = dtestr

  #Case Processor part
  text_cp = sent_tokenize(df.iloc[i,12])
  spark_df_cp = spark.createDataFrame(text_cp, StringType()).toDF("text")
  result_cp = pipl.fit(spark_df_cp).transform(spark_df_cp)
  resultdf_cp = result_cp.toPandas()
  for j in resultdf_cp['date']:
    if j != []:
      for k in j:
        dtestr=dtestr+str(k.result)+", "
  df.iloc[i,14]+=dtestr

In [122]:
df

,Number,External Info,ORT manual,Manager,Assigned to,State,Priority,Next SLA due date,Next SLA reason,Tags,Updated by,Customer contact,CP_ext_inf,OA_ext_inf,dates,check
0,CS20240007061863,2024-03-22 21:54:08 - Aaron Cooke (I565751) (E...,2024-03-25 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),Awaiting Info,2 - High,2024-03-25 22:30:00,ORT (SLA),NaN,S0023855138,Swapna Shetty S (S0021840755),. Hi there. Investigation is still continuing ...,. You will receive an update about this incide...,"2024.03.18, , , 2024.03.15, , , 2024.02.23, , ...",
1,CS20240007164010,2024-03-25 14:51:03 - Aaron Cooke (I565751) (E...,2024-03-26 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,2 - High,2024-03-26 22:30:00,ORT (SLA),NaN,system,Arun Kumara Y A (S0018363415),. Hi Arun. Thank you for this - the colleague ...,. You will receive an update about this incide...,"2024.03.25, , , 2024.03.22, , , 2024.03.11, , ...",
2,CS20240007084283,2024-03-23 01:29:03 - Lovely Barba (I349243) (...,2024-03-27 03:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,1 - Very High,2024-03-27 03:30:00,ORT (SLA),NaN,I545965,marychu billones (S0025814704),". Hi Marychu,. Good day!. My name is Lovely an...",. . Contact phone or email: +95-17301589.,"2024.03.25, 2024.01.05, 2024.01.29, 2024.03.25...",
3,CS20230006469172,2024-03-22 18:58:47 - Aaron Cooke (I565751) (E...,2024-03-27 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,3 - Medium,2024-03-27 22:30:00,ORT (SLA),"#PFA_90+_Bustathon_2024, #PL_HELP_Tobias, #Q1_...",system,Lisa Dickie (S0025121929),. Hi there. I have brought to product lead tod...,. . Channel: Call from customer. Interaction s...,"2024.03.25, 2024.03.26, 2024.03.25,",
4,CS20240007189121,2024-03-21 15:03:22 - Aaron Cooke (I565751) (E...,2024-03-27 22:30:00,Marie Hardiman (I322855),Aaron Cooke (I565751),In progress,2 - High,2024-03-27 22:30:00,ORT (SLA),NaN,system,Erin Dick (S0014020613),. Hi Erin. The colleague who I was advised to ...,. You will receive an update about this incide...,"2024.03.12, , , 2024.03.08, , , 2024.03.07, , ...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,CS20230006603345,2024-02-16 06:01:05 - Youssouf Sangare (I31938...,2024-03-10 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,3 - Medium,2024-03-10 05:29:56,ORT (SLA),"#PFA_90+_Bustathon_2024, #Q1_2024_Backlog_Trac...",S0023223396,Ulrike Stadler (S0023223396),". &nbsp;Hello Sabrina ,. your feedback are und...",. . Contact phone or email: +353-831817719.,"2024.03.25, 2024.03.25, 2024.03.25, 2023.12.19...",
463,CS20240007008827,2024-03-18 07:01:34 - Youssouf Sangare (I31938...,2024-04-06 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,4 - Low,2024-04-06 05:29:56,ORT (SLA),NaN,system,Nina Hynynen (S0021536958),". Hi Nina,. we are working on your feedback an...",,"2024.02.22, 2222.01.11,",
464,CS20240007270560,2024-03-23 02:23:25 - Youssouf Sangare (I31938...,2024-04-06 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,2 - High,2024-04-06 05:29:56,ORT (SLA),NaN,system,Ahmed haridy (S0024058876),". Hello &nbsp;Ahmed &nbsp; ,&nbsp;. Call feedb...",. . Contact phone or email: +20-01030554221.,"2024.03.25, 2570.01.03, 2024.03.25, 2024.03.25...",
465,CS20240006832810,2024-03-20 18:42:14 - Chloe Doyle (I530079) (E...,2024-04-01 05:29:56,Liam Hudson (I323795),Youssouf Sangare (I319380),In progress,3 - Medium,2024-04-01 05:29:56,ORT (SLA),#Q1_2024_Backlog_Tracking,I530079,Michelle McCann (S0025036898),. <b>Requester name:</b> Roy SAP<br /><b>Reque...,. . Channel: Call from customer. Interaction s...,,


In [123]:
count=0
for i in range(0,len(df)-1):
  omd = word_tokenize(str(df.iloc[i,2]))
  omdte = convert_dte(omd[0])
  if len(str(omdte))>5:
    for j in df.iloc[i,14].split(","):
      if str(omdte)==str(j):
        df.iloc[i,15]='ORTM_OK'
        count=count+1
        continue
      else:
        pass
print(count)

51


In [114]:
df

,Case,External Info,ORT manual,Updated by,Customer contact,Company,Source channel Task,State,Priority,Assigned to,Updated,Tags,CP_ext_inf,OA_ext_inf,dates,check
0,API user restriction​CS20230006607972,2023-12-20 12:09:32 - Rohit Panwar (I322661) (...,2023-12-20 15:00:35,I322661,Anitha Ponna (S0025152086),NaN,Chat,Resolved,3 - Medium,Rohit Panwar (I322661),2023-12-20 12:09:32,$ORT_Manual_OK,". Dear Anitha,. Thank you for your response. T...",. You will receive an update about this incide...,"2023.12.20, , , 2023.12.07, , , 2023.12.06, , ...",ORTM_OK
1,Users not imported from SF into LMS​CS20230006...,2023-12-15 04:12:16 - System (External Info)\n...,2023-12-15 15:00:31,I322661,Dominic Ponnuswamy (S0024832552),NaN,Portal,Closed,2 - High,Rohit Panwar (I322661),2023-12-18 10:35:06,"$ORT_Manual_OK, @Not_Release, IRT_METBY_QG",". Dear Dominic,. I have looked into the integr...",. You will receive an update about this incide...,"2023.12.15, , , 2023.12.15, , , 2024.03.25, 20...",ORTM_OK
2,WFA shows no pending reqs but there are pendin...,2023-12-27 05:18:08 - System (External Info)\n...,2023-12-06 20:41:39,I501666,Mandy Birk (S0016457108),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:01:32,"$ORT_Manual_NOK, IRT_METBY_POC",". Hi Mandy,. As requested plesae provide the d...",. You will receive an update about this incide...,"2023.11.28, , , 2023.11.28, , ,",
3,Can't activate Onboarding 2.0 in the Productio...,2023-12-01 08:26:46 - Subba Reddy Venkata Takk...,2023-12-13 02:35:21,I501666,Alfonso Adeva (S0024115440),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-05 11:58:27,$ORT_Manual_NOK,". Hi Customer,. Corrected config related to er...",,,
4,Not able to add input control for background e...,2024-01-01 07:04:12 - Subba Reddy Venkata Takk...,2023-12-31 18:06:09,I501666,Aji Hussain (S0021762292),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-01 07:04:12,$ORT_Manual_NOK,. Solution provided:[code]. As of now closing ...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Received an error when importing a story​CS202...,2023-12-18 10:31:14 - Subba Reddy Venkata Takk...,2023-12-21 20:30:00,system,Kumi Mikami (S0024714497),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-28 08:52:48,$ORT_Manual_NOK,". Thanks for the patience Kumi,. Engineering t...",. . Channel: Call from customer. Interaction s...,"2024.01.12, 2024.03.25, 2024.03.25, 2024.01.12...",
113,"Everytime I try to run a story report, the err...",2024-01-09 05:51:25 - System (External Info)\n...,2023-12-19 20:58:38,system,Park Sujeong (S0024711903),NaN,Portal,Closed,2 - High,Subba Reddy Venkata Takkoli (I501666),2024-01-09 05:51:25,$ORT_Manual_NOK,". Hi,. I am available for call in any feasible...",. You will receive an update about this incide...,"2023.12.11, , , 2023.12.11, , , 2024.03.25,",
114,ORD Report Error​CS20230006745117,2023-12-21 20:47:58 - Subba Reddy Venkata Takk...,2023-12-22 13:33:58,system,Jianliang Yin (S0019691286),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-30 09:34:40,$ORT_Manual_NOK,". Thanks, as of now closing the case. Looks li...",,,
115,An error occurred while importing a Story repo...,2023-12-28 06:03:25 - Subba Reddy Venkata Takk...,2023-12-31 12:54:39,system,Napapatch Rakwong (S0025555256),NaN,Portal,Resolved,2 - High,Subba Reddy Venkata Takkoli (I501666),2023-12-29 08:57:03,$ORT_Manual_NOK,. Solution provided:[code]. &nbsp;. Issue is f...,,,


In [124]:
import openpyxl
df.to_excel('spark.xlsx', sheet_name='dates form spark')